In [1]:
import sys
sys.path.append("..") # To get storm code

In [2]:
from src.weatherboy import *
from src.db import *

## Training Population (Simulated Runs)
The fake runner can be used to perform run simulations against the database. This is not connected to Spotify, only the storm backend you've created thus far. When the storm is freshly configured it doesn't have a consistent history to model on, so the fake runner will build out a sample population under one large assumption: <b>You've heard every track by every artist in your targets and have picked only the ones you like.</b> This is likely false, but so long as it is directionally true it should be sufficient.

In [7]:
fr = FakeRunner('film_vg_instrumental', '2020-01-07', '2020-01-14')
fr_tracks = fr.Run()

NameError: name 'FakeRunner' is not defined

## Pipeline and model
The pipeline from the backend to the modeling ecosystem. Using a set of tracks as your total population, the pipeline serves as a mechanism for obtaining features for every track (including the target variables). As artist-level and album-level features get added they will be sourced through this mechanism as well. Additionally, as is important for data-centrism, the pipeline configuration is stored alongside the model, meaning a model is both a <b>pipeline+method</b>

Since the fakerunner has already run, the modeling component only exists beneath these steps. Each model and pipeline tried will be automatically registered. Registry contains the method and pipeline information needed to evaluate the model. All registered models can be evaluated side by side. If there is a model that does particularly well you can prdocutionalize it by calling:

```WeatherBoyPipeline().move_model_to_production(model_name, storm_name)```

This will add the model into the storm's configuration and will now be leveraged on subsequent runs.

In [ ]:
# Load a pipeline
wbp = WeatherBoyPipeline(fr_tracks['storm_tracks'], 
                         storm_name='film_vg_instrumental',
                         features={'_id':0, 
                                   'album_id':0, 
                                   'last_updated':0,
                                   'name':0,
                                   'time_signature':0,
                                   'artists':0},
                        train=True)
wbp.load()

In [6]:
wbm = WeatherBoyModel(output_dir='../models')

[]